In [46]:
import os
os.environ['GROQ_API_KEY'] = ''

In [38]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 46.8 MB/s eta 0:00:00


In [63]:
from langchain_groq import ChatGroq



In [28]:
llm=ChatGroq(model="Gemma2-9b-It")

In [29]:
from langchain_core.messages import HumanMessage
llm.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hello Krish, it's nice to meet you! \n\nThat's a very impressive title.  What kind of AI work are you involved with? I'm always eager to learn more about the exciting things people are doing with artificial intelligence. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 22, 'total_tokens': 79, 'completion_time': 0.103636364, 'prompt_time': 0.00014876, 'queue_time': 0.13877467400000001, 'total_time': 0.103785124}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0a879d61-bc0f-4377-8574-a6e7ff0b8f9e-0', usage_metadata={'input_tokens': 22, 'output_tokens': 57, 'total_tokens': 79})

In [30]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [ ]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=bdb2cadbec58d9659e17120427d44cded8303aa142361eb3792e918a050f9943
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [31]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [32]:
!pip install arxiv


In [33]:
api_wrapper_arxvi=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxvi=ArxivQueryRun(api_wrapper=api_wrapper_arxvi)
arxvi.name

'arxiv'

In [16]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [20]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
documents


[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transforma

In [50]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,hf_embeddings)
retriever=vectordb.as_retriever()

retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7c6d3f984550>, search_kwargs={})

In [51]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")

retriever_tool.name

'langsmith-search'

In [52]:
tools=[arxvi,wiki,retriever_tool]

In [54]:
from langchain.prompts import PromptTemplate

# Define your custom prompt
template = """
You are a helpful assistant. Answer the following question concisely:
{question}
"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

# Example of using the prompt
formatted_prompt = prompt.format(question="What is langsmith?")



In [56]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [57]:
from langchain.agents import AgentExecutor, create_react_agent

In [58]:
agent=create_react_agent(llm,tools,prompt)

In [59]:
agentexecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [62]:
a=agentexecutor.      invoke({"input":"what is ML"})
a

{'input': 'what is ML',
 'output': 'Machine Learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks with minimal human intervention.'}